In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [160]:
start_osm = 310280534
mid_osm = []
end_osm = 310280538
start_mesh = 50325671212
end_mesh = 50325662413

In [161]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [162]:
getdbname = "kddi_18_new_param"
dbname = "kddi_trkn18_monthly_anbun_4c_32"
trknName = "trkn_18_grp"
sttime = "2022-04-01 18:00:00"

In [163]:
datedir = "230225_1"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        #cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        #cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        #cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            #cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            #cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        #cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        if dis_km == 0:
            dte = dte + datetime.timedelta(minutes=15) 
            continue

        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値27.05距離0.0 km
存在メッシュ：50325671221値38.333381639975286距離0.14467375091535445 km
存在メッシュ：50325661434値36.0612263870126距離0.11554053238239893 km
存在メッシュ：50325672211値106.03367144303692距離1.0127162559357643 km
存在メッシュ：50325672122値94.75028981690758距離0.8680425051979405 km
存在メッシュ：50325671222値49.61676327910186距離0.2893475018198268 km
存在メッシュ：50325671221値38.333381639975286距離0.14467375091535445 km
存在メッシュ：50325672111値60.90014491714676距離0.4340212527104301 km
存在メッシュ：50325672121値83.46690818667595距離0.7233687544075176 km
存在メッシュ：50325672112値72.18352655326126距離0.578695003576282 km
存在メッシュ：50325671212値27.05距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値28.166666666666668距離0.0 km
存在メッシュ：50325671221値39.29461741561415距離0.14467375091535445 km
存在メッシュ：50325661434値37.053761566215925距離0.11554053238239893 km
存在メッシュ：50325672211値106.06232187301572距離1.0127162559357643 km
存在メッシュ：50325672122値94.93437113772346距離0.86804250

存在メッシュ：50325671212値11.816666666666666距離0.0 km
存在メッシュ：50325671221値16.57195088347398距離0.14467375091535445 km
存在メッシュ：50325661434値15.614370580218992距離0.11554053238239893 km
存在メッシュ：50325672211値45.10365616881299距離1.0127162559357643 km
存在メッシュ：50325672122値40.34837195784094距離0.8680425051979405 km
存在メッシュ：50325671222値21.327235099923612距離0.2893475018198268 km
存在メッシュ：50325671221値16.57195088347398距離0.14467375091535445 km
存在メッシュ：50325672111値26.08251931591738距離0.4340212527104301 km
存在メッシュ：50325672121値35.593087745140004距離0.7233687544075176 km
存在メッシュ：50325672112値30.837803531097595距離0.578695003576282 km
存在メッシュ：50325671212値11.816666666666666距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値11.833333333333334距離0.0 km
存在メッシュ：50325671221値16.289018876130523距離0.14467375091535445 km
存在メッシュ：50325661434値15.39176930728241距離0.11554053238239893 km
存在メッシュ：50325672211値43.02313211838564距離1.0127162559357643 km
存在メッシュ：50325672122値38.56744658105607距離0.8680425051979405 km
存在メッシュ：5032567122

33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値28.166666666666668距離0.0 km
存在メッシュ：50325671221値39.29461741561415距離0.14467375091535445 km
存在メッシュ：50325661434値37.053761566215925距離0.11554053238239893 km
存在メッシュ：50325672211値106.06232187301572距離1.0127162559357643 km
存在メッシュ：50325672122値94.93437113772346距離0.8680425051979405 km
存在メッシュ：50325671222値50.422568163724605距離0.2893475018198268 km
存在メッシュ：50325671221値39.29461741561415距離0.14467375091535445 km
存在メッシュ：50325672111値61.55051891076829距離0.4340212527104301 km
存在メッシュ：50325672121値83.80642039838541距離0.7233687544075176 km
存在メッシュ：50325672112値72.67846965590815距離0.578695003576282 km
存在メッシュ：50325671212値28.166666666666668距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値26.4距離0.0 km
存在メッシュ：50325671221値37.17879439953718距離0.14467375091535445 km
存在メッシュ：50325661434値35.008248804522914距離0.11554053238239893 km
存在メッシュ：50325672211値101.85156076161542距離1.0127162559357643 km
存在メッシュ：50325672122値91.0727663753050

33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値10.116666666666667距離0.0 km
存在メッシュ：50325671221値14.892224477955871距離0.14467375091535445 km
存在メッシュ：50325661434値13.930561643801168距離0.11554053238239893 km
存在メッシュ：50325672211値43.54557133012011距離1.0127162559357643 km
存在メッシュ：50325672122値38.77001352469104距離0.8680425051979405 km
存在メッシュ：50325671222値19.66778228888586距離0.2893475018198268 km
存在メッシュ：50325671221値14.892224477955871距離0.14467375091535445 km
存在メッシュ：50325672111値24.443340099358053距離0.4340212527104301 km
存在メッシュ：50325672121値33.994455717525724距離0.7233687544075176 km
存在メッシュ：50325672112値29.218897909013215距離0.578695003576282 km
存在メッシュ：50325671212値10.116666666666667距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値9.95距離0.0 km
存在メッシュ：50325671221値13.783961978685953距離0.14467375091535445 km
存在メッシュ：50325661434値13.011910024095716距離0.11554053238239893 km
存在メッシュ：50325672211値36.78773383830083距離1.0127162559357643 km
存在メッシュ：50325672122値32.9537718643

存在メッシュ：50325671212値26.4距離0.0 km
存在メッシュ：50325671221値37.17879439953718距離0.14467375091535445 km
存在メッシュ：50325661434値35.008248804522914距離0.11554053238239893 km
存在メッシュ：50325672211値101.85156076161542距離1.0127162559357643 km
存在メッシュ：50325672122値91.07276637530501距離0.8680425051979405 km
存在メッシュ：50325671222値47.9575887982636距離0.2893475018198268 km
存在メッシュ：50325671221値37.17879439953718距離0.14467375091535445 km
存在メッシュ：50325672111値58.736383195956726距離0.4340212527104301 km
50325672121 is kddi none!
存在メッシュ：50325672112値69.51517759180577距離0.578695003576282 km
存在メッシュ：50325671212値26.4距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値25.183333333333334距離0.0 km
存在メッシュ：50325671221値35.00025830578538距離0.14467375091535445 km
存在メッシュ：50325661434値33.02340612123529距離0.11554053238239893 km
存在メッシュ：50325672211値93.90180810848904距離1.0127162559357643 km
存在メッシュ：50325672122値84.08488314808343距離0.8680425051979405 km
存在メッシュ：50325671222値44.81718327749901距離0.2893475018198268 km
存在メッシュ：50325671221値35.

...

33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値11.833333333333334距離0.0 km
50325671221 is kddi none!
50325661434 is kddi none!
存在メッシュ：50325672211値43.02313211838564距離1.0127162559357643 km
存在メッシュ：50325672122値38.56744658105607距離0.8680425051979405 km
存在メッシュ：50325671222値20.744704418592562距離0.2893475018198268 km
50325671221 is kddi none!
存在メッシュ：50325672111値25.20038996062746距離0.4340212527104301 km
存在メッシュ：50325672121値34.111761042106544距離0.7233687544075176 km
存在メッシュ：50325672112値29.656075501900062距離0.578695003576282 km
存在メッシュ：50325671212値11.833333333333334距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値10.116666666666667距離0.0 km
存在メッシュ：50325671221値14.892224477955871距離0.14467375091535445 km
50325661434 is kddi none!
存在メッシュ：50325672211値43.54557133012011距離1.0127162559357643 km
存在メッシュ：50325672122値38.77001352469104距離0.8680425051979405 km
存在メッシュ：50325671222値19.66778228888586距離0.2893475018198268 km
存在メッシュ：50325671221値14.892224477955871距離0.1

33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値26.4距離0.0 km
存在メッシュ：50325671221値37.17879439953718距離0.14467375091535445 km
存在メッシュ：50325661434値35.008248804522914距離0.11554053238239893 km
存在メッシュ：50325672211値101.85156076161542距離1.0127162559357643 km
存在メッシュ：50325672122値91.07276637530501距離0.8680425051979405 km
存在メッシュ：50325671222値47.9575887982636距離0.2893475018198268 km
存在メッシュ：50325671221値37.17879439953718距離0.14467375091535445 km
存在メッシュ：50325672111値58.736383195956726距離0.4340212527104301 km
存在メッシュ：50325672121値80.29397198507576距離0.7233687544075176 km
存在メッシュ：50325672112値69.51517759180577距離0.578695003576282 km
存在メッシュ：50325671212値26.4距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値25.183333333333334距離0.0 km
存在メッシュ：50325671221値35.00025830578538距離0.14467375091535445 km
存在メッシュ：50325661434値33.02340612123529距離0.11554053238239893 km
存在メッシュ：50325672211値93.90180810848904距離1.0127162559357643 km
存在メッシュ：50325672122値84.08488314808343距離0.868042505197

33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値9.95距離0.0 km
存在メッシュ：50325671221値13.783961978685953距離0.14467375091535445 km
存在メッシュ：50325661434値13.011910024095716距離0.11554053238239893 km
50325672211 is kddi none!
存在メッシュ：50325672122値32.95377186431957距離0.8680425051979405 km
50325671222 is kddi none!
存在メッシュ：50325671221値13.783961978685953距離0.14467375091535445 km
存在メッシュ：50325672111値21.451885935113555距離0.4340212527104301 km
存在メッシュ：50325672121値29.119809888944395距離0.7233687544075176 km
存在メッシュ：50325672112値25.285847912487654距離0.578695003576282 km
存在メッシュ：50325671212値9.95距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値8.516666666666667距離0.0 km
存在メッシュ：50325671221値12.154650565361036距離0.14467375091535445 km
存在メッシュ：50325661434値11.422063076134695距離0.11554053238239893 km
存在メッシュ：50325672211値33.982553945665394距離1.0127162559357643 km
存在メッシュ：50325672122値30.344570051435234距離0.8680425051979405 km
存在メッシュ：50325671222値15.792634463781761距離0.289347501819

33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値22.65距離0.0 km
存在メッシュ：50325671221値32.09298978534167距離0.14467375091535445 km
存在メッシュ：50325661434値30.19143761516407距離0.11554053238239893 km
存在メッシュ：50325672211値88.75092846660229距離1.0127162559357643 km
存在メッシュ：50325672122値79.3079386928482距離0.8680425051979405 km
存在メッシュ：50325671222値41.53597956997305距離0.2893475018198268 km
存在メッシュ：50325671221値32.09298978534167距離0.14467375091535445 km
存在メッシュ：50325672111値50.978969353699185距離0.4340212527104301 km
存在メッシュ：50325672121値69.86494891566093距離0.7233687544075176 km
存在メッシュ：50325672112値60.42195913580978距離0.578695003576282 km
存在メッシュ：50325671212値22.65距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値22.383333333333333距離0.0 km
存在メッシュ：50325671221値31.11224206859072距離0.14467375091535445 km
存在メッシュ：50325661434値29.354485708991913距離0.11554053238239893 km
存在メッシュ：50325672211値83.48569445167394距離1.0127162559357643 km
存在メッシュ：50325672122値74.75678572712789距離0.86804250519

33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値8.25距離0.0 km
存在メッシュ：50325671221値11.457733171356521距離0.14467375091535445 km
存在メッシュ：50325661434値10.811786060112986距離0.11554053238239893 km
存在メッシュ：50325672211値30.70413218903665距離1.0127162559357643 km
存在メッシュ：50325672122値27.496399021616373距離0.8680425051979405 km
存在メッシュ：50325671222値14.665466342471762距離0.2893475018198268 km
存在メッシュ：50325671221値11.457733171356521距離0.14467375091535445 km
存在メッシュ：50325672111値17.873199513279495距離0.4340212527104301 km
存在メッシュ：50325672121値24.28866585302986距離0.7233687544075176 km
存在メッシュ：50325672112値21.080932683538435距離0.578695003576282 km
存在メッシュ：50325671212値8.25距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
存在メッシュ：50325671212値6.85距離0.0 km
存在メッシュ：50325671221値9.593693119882191距離0.14467375091535445 km
存在メッシュ：50325661434値9.041190604787651距離0.11554053238239893 km
存在メッシュ：50325672211値26.05585183022938距離1.0127162559357643 km
存在メッシュ：50325672122値23.312158713714距離0.8680425051979405 km
存在メ

In [164]:
connection.close()

In [149]:
#CreateGeojsonFile